In [1]:
from loguru import logger
import dateutil
import datetime
import numpy as np
import os

In [2]:
datetime.timedelta(0)

datetime.timedelta(0)

In [5]:
def cal_repeat_actions(prefix, els=''):
    print(prefix)
    pattern = r"(?P<time>.*) \| (?P<level>.*)     \| (?P<message>.*)"  # Regex with named groups
    caster_dict = dict(time=dateutil.parser.parse, level=str, message=str)        # Transform matching groups

    actions = []
    groups = list(logger.parse(prefix+"/fixed_in_site/"+els+"experiment_record.log", pattern, cast=caster_dict))
    for ind, group in enumerate(groups):
        if group['message'].startswith('daifu.repair:do') and groups[ind+1]['message'].startswith('daifu.repair:do'):
            actions.append([group['message'].split('... ')[-1], groups[ind+1]['message'].split('... ')[-1]])
        elif group['message'].startswith('daifu.repair:do') and groups[ind-1]['message'].startswith('daifu.repair:do'):
            continue
        elif group['message'].startswith('daifu.repair:do'):
            actions.append(group['message'].split('... ')[-1])
    print(actions)

In [6]:
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/009")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/016", 'gated_pixelcnn/')
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/018")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/028", 'cnp/')
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/029")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/039")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/057")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/058")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/079")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/096")
cal_repeat_actions("../DLFailureBenchmark/API_Misuse/ggnn.pytorch", 'ggnn.pytorch/')
cal_repeat_actions("../DLFailureBenchmark/Tensor_Mismatch/007", 'cifar10/')
cal_repeat_actions("../DLFailureBenchmark/Tensor_Mismatch/036")
cal_repeat_actions("../DLFailureBenchmark/Tensor_Mismatch/056")
cal_repeat_actions("../DLFailureBenchmark/Tensor_Mismatch/080")
cal_repeat_actions("../DLFailureBenchmark/Tensor_Mismatch/081")
cal_repeat_actions("../DLFailureBenchmark/Tensor_Mismatch/090")
cal_repeat_actions("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-1")
cal_repeat_actions("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-1")
cal_repeat_actions("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-1")
cal_repeat_actions("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-2")
cal_repeat_actions("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-2")
cal_repeat_actions("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-2")
cal_repeat_actions("../DLFailureBenchmark/Path_Not_Found/CV-PNF")
cal_repeat_actions("../DLFailureBenchmark/Path_Not_Found/NLP-PNF")
cal_repeat_actions("../DLFailureBenchmark/Path_Not_Found/RL-PNF")

../DLFailureBenchmark/API_Misuse/009
['Action', 'Surgery', 'Surgery', 'Surgery']
../DLFailureBenchmark/API_Misuse/016
['Surgery']
../DLFailureBenchmark/API_Misuse/018
[['Action', 'Surgery']]
../DLFailureBenchmark/API_Misuse/028
['Action']
../DLFailureBenchmark/API_Misuse/029
['Surgery']
../DLFailureBenchmark/API_Misuse/039
['Surgery']
../DLFailureBenchmark/API_Misuse/057
['Action']
../DLFailureBenchmark/API_Misuse/058
['Action']
../DLFailureBenchmark/API_Misuse/079
['Action']
../DLFailureBenchmark/API_Misuse/096
['Surgery']
../DLFailureBenchmark/API_Misuse/ggnn.pytorch
['Surgery']
../DLFailureBenchmark/Tensor_Mismatch/007
['Action']
../DLFailureBenchmark/Tensor_Mismatch/036
['Surgery', 'Action']
../DLFailureBenchmark/Tensor_Mismatch/056
['Action']
../DLFailureBenchmark/Tensor_Mismatch/080
['Action']
../DLFailureBenchmark/Tensor_Mismatch/081
['Surgery', 'Action']
../DLFailureBenchmark/Tensor_Mismatch/090
['Surgery', 'Action', 'Surgery']
../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-1
[

In [7]:
def cal_benefit_repeated_experiments(prefix, els=''):
    print(prefix)
    pattern = r"(?P<time>.*) \| (?P<level>.*)     \| (?P<message>.*)"  # Regex with named groups
    caster_dict = dict(time=dateutil.parser.parse, level=str, message=str)        # Transform matching groups

    files_name = os.listdir(prefix+"/fixed_in_site_repeated_experiments/"+els)
    repeat_records = [s for s in files_name if s.startswith('experiment_record_')]

    per_time_total = []
    per_time_waste = []
    for record in repeat_records:
        groups = []
        for group in logger.parse(prefix+"/fixed_in_site_repeated_experiments/"+els+record, pattern, cast=caster_dict):
            groups.append(group)
        time_total = groups[-1]['time'] - groups[0]['time']
        waste_time = datetime.timedelta(0)
        for i, group in enumerate(groups):
            if i >= 3 and i%3==0:
                waste_time += groups[i-2]['time'] - groups[0]['time']
        per_time_total.append(time_total)
        per_time_waste.append(waste_time)

    
    assert len(per_time_total) == 10
    assert len(per_time_waste) == 10

    time_total = np.mean(per_time_total)
    time_waste = np.mean(per_time_waste)
    benefit = time_waste/time_total
    print('gained benefit:', np.round(benefit*100, 2), '%')
    #print('daifu number of fix :', numberfix)

    
    return benefit

In [8]:
benefits = []
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/009"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/016", 'gated_pixelcnn/'))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/018"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/028", 'cnp/'))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/029"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/039"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/057"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/058"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/079"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/096"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/API_Misuse/ggnn.pytorch", 'ggnn.pytorch/'))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/007", 'cifar10/'))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/036"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/056"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/080"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/081"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/090"))
#benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-1"))
#benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-1"))
#benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-1"))
#benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-2"))
#benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-2"))
#benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-2"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Path_Not_Found/CV-PNF"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Path_Not_Found/NLP-PNF"))
benefits.append(cal_benefit_repeated_experiments("../DLFailureBenchmark/Path_Not_Found/RL-PNF"))

../DLFailureBenchmark/API_Misuse/009
gained benefit: 1.65 %
../DLFailureBenchmark/API_Misuse/016
gained benefit: 99.83 %
../DLFailureBenchmark/API_Misuse/018
gained benefit: 2.64 %
../DLFailureBenchmark/API_Misuse/028
gained benefit: 0.14 %
../DLFailureBenchmark/API_Misuse/029
gained benefit: 96.1 %
../DLFailureBenchmark/API_Misuse/039
gained benefit: 1.06 %
../DLFailureBenchmark/API_Misuse/057
gained benefit: 3.47 %
../DLFailureBenchmark/API_Misuse/058
gained benefit: 4.5 %
../DLFailureBenchmark/API_Misuse/079
gained benefit: 7.18 %
../DLFailureBenchmark/API_Misuse/096
gained benefit: 51.33 %
../DLFailureBenchmark/API_Misuse/ggnn.pytorch
gained benefit: 28.47 %
../DLFailureBenchmark/Tensor_Mismatch/007
gained benefit: 35.4 %
../DLFailureBenchmark/Tensor_Mismatch/036
gained benefit: 0.98 %
../DLFailureBenchmark/Tensor_Mismatch/056
gained benefit: 1.24 %
../DLFailureBenchmark/Tensor_Mismatch/080
gained benefit: 0.78 %
../DLFailureBenchmark/Tensor_Mismatch/081
gained benefit: 0.63 %
../D

In [9]:
def cal_time_repeated_experiments(prefix, els=''):
    print(prefix)
    pattern = r"(?P<time>.*) \| (?P<level>.*)     \| (?P<message>.*)"  # Regex with named groups
    caster_dict = dict(time=dateutil.parser.parse, level=str, message=str)        # Transform matching groups

    files_name = os.listdir(prefix+"/fixed_in_site_repeated_experiments/"+els)
    repeat_records = [s for s in files_name if s.startswith('experiment_record_')]

    per_time_daifu = []
    for record in repeat_records:
        groups = []
        for group in logger.parse(prefix+"/fixed_in_site_repeated_experiments/"+els+record, pattern, cast=caster_dict):
            groups.append(group)
        total_time_daifu_include_waiting = groups[-1]['time'] - groups[0]['time']
        waiting_time = datetime.timedelta(0)
        numberfix = 0
        for i, group in enumerate(groups):
            if i >= 3 and i%3==0:
                numberfix += 1
                waiting_time += groups[i]['time'] - groups[i-2]['time']
        # since processed automatically in repeated experiments, so don't need to exclude human operation time
        time_daifu = total_time_daifu_include_waiting# - waiting_time
        #print('daifu total:', total_time_daifu_include_waiting)
        #print('daifu waiting:', waiting_time)
        #print('daifu actual:', time_daifu)
        per_time_daifu.append(time_daifu)

    files_name = os.listdir(prefix+"/fixed_repeated_experiments/"+els)
    repeat_records = [s for s in files_name if s.startswith('experiment_record_')]

    per_total_time_origin = []
    for record in sorted(repeat_records):
        groups = []
        for group in logger.parse(prefix+"/fixed_repeated_experiments/"+els+record, pattern, cast=caster_dict):
            groups.append(group)
        total_time_origin = groups[-1]['time'] - groups[0]['time']
        per_total_time_origin.append(total_time_origin)
        #print('origin actual:', total_time_origin)
    
    assert len(per_time_daifu) == 10
    assert len(per_total_time_origin) == 10

    time_daifu = np.mean(per_time_daifu)
    total_time_origin = np.mean(per_total_time_origin)
    extra_overhead = (time_daifu-total_time_origin)/total_time_origin
    print('extra overhead:', np.round(extra_overhead*100, 2), '%')
    #print('daifu number of fix :', numberfix)
    #print('extra overhead:', extra_overhead)

    
    return extra_overhead

In [10]:
extra_overheads = []
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/009"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/016", 'gated_pixelcnn/'))

extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/018"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/028", 'cnp/'))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/029"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/039"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/057"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/058"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/079"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/096"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/API_Misuse/ggnn.pytorch", 'ggnn.pytorch/'))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/007", 'cifar10/'))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/036"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/056"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/080"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/081"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Tensor_Mismatch/090"))
# extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-1"))
# extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-1"))
# extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-1"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-2"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-2"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-2"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Path_Not_Found/CV-PNF"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Path_Not_Found/NLP-PNF"))
extra_overheads.append(cal_time_repeated_experiments("../DLFailureBenchmark/Path_Not_Found/RL-PNF"))

../DLFailureBenchmark/API_Misuse/009
extra overhead: -0.02 %
../DLFailureBenchmark/API_Misuse/016
extra overhead: 0.14 %
../DLFailureBenchmark/API_Misuse/018
extra overhead: 1.8 %
../DLFailureBenchmark/API_Misuse/028
extra overhead: -1.57 %
../DLFailureBenchmark/API_Misuse/029
extra overhead: 4.36 %
../DLFailureBenchmark/API_Misuse/039
extra overhead: -0.33 %
../DLFailureBenchmark/API_Misuse/057
extra overhead: 2.32 %
../DLFailureBenchmark/API_Misuse/058
extra overhead: 4.06 %
../DLFailureBenchmark/API_Misuse/079
extra overhead: 1.86 %
../DLFailureBenchmark/API_Misuse/096
extra overhead: 1.66 %
../DLFailureBenchmark/API_Misuse/ggnn.pytorch
extra overhead: 6.73 %
../DLFailureBenchmark/Tensor_Mismatch/007
extra overhead: 2.58 %
../DLFailureBenchmark/Tensor_Mismatch/036
extra overhead: -2.35 %
../DLFailureBenchmark/Tensor_Mismatch/056
extra overhead: 0.36 %
../DLFailureBenchmark/Tensor_Mismatch/080
extra overhead: 0.2 %
../DLFailureBenchmark/Tensor_Mismatch/081
extra overhead: 1.52 %
../D

In [11]:
print('min:', np.min(extra_overheads))
print('max:', np.max(extra_overheads))
print('mean:', np.mean(extra_overheads))

min: -0.023807099010548543
max: 0.0672736469398078
mean: 0.01103367575924449


In [12]:
def cal_quality_measures(prefix, els='', origin=False):
    print(prefix)
    pattern = r"(?P<time>.*) \| (?P<level>.*)     \| (?P<message>.*)"  # Regex with named groups
    caster_dict = dict(time=dateutil.parser.parse, level=str, message=str)        # Transform matching groups

    files_name = os.listdir(prefix+"/fixed_in_site_repeated_experiments/"+els)
    repeat_records = [s for s in files_name if s.startswith('experiment_record_')]

    per_qm_daifu = []
    for record in repeat_records:
        groups = []
        for group in logger.parse(prefix+"/fixed_in_site_repeated_experiments/"+els+record, pattern, cast=caster_dict):
            groups.append(group)
        if origin:
            per_qm_daifu.append(' '.join(groups[-1]['message'].split(' ')[7:]))
        else:
            per_qm_daifu.append(float(groups[-1]['message'].split(' ')[-1]))

    files_name = os.listdir(prefix+"/fixed_repeated_experiments/"+els)
    repeat_records = [s for s in files_name if s.startswith('experiment_record_')]

    per_qm_origin = []
    for record in repeat_records:
        groups = []
        for group in logger.parse(prefix+"/fixed_repeated_experiments/"+els+record, pattern, cast=caster_dict):
            groups.append(group)
        if origin:
            per_qm_origin.append(' '.join(groups[-1]['message'].split(' ')[7:]))
        else:
            per_qm_origin.append(float(groups[-1]['message'].split(' ')[-1]))

    return per_qm_daifu, per_qm_origin

In [13]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import matplotlib.pyplot as plt

from patsy import dmatrices
import statsmodels.stats.power as pw
from scipy.stats import wilcoxon
from statsmodels.tools.sm_exceptions import PerfectSeparationError



#calculates cohen's kappa value
def cohen_d(orig_accuracy_list, accuracy_list):
    nx = len(orig_accuracy_list)
    ny = len(accuracy_list)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx-1)*np.std(orig_accuracy_list, ddof=1) ** 2 + (ny-1)*np.std(accuracy_list, ddof=1) ** 2) / dof)
    result = (np.mean(orig_accuracy_list) - np.mean(accuracy_list)) / pooled_std
    return result


def p_value_wilcoxon(orig_accuracy_list, accuracy_list):
    w, p_value_w = wilcoxon(orig_accuracy_list, accuracy_list)

    return p_value_w


def p_value_glm(orig_accuracy_list, accuracy_list):
    list_length = len(orig_accuracy_list)

    zeros_list = [0] * list_length
    ones_list = [1] * list_length
    mod_lists = zeros_list + ones_list
    acc_lists = orig_accuracy_list + accuracy_list

    data = {'Acc': acc_lists, 'Mod': mod_lists}
    df = pd.DataFrame(data)

    response, predictors = dmatrices("Acc ~ Mod", df, return_type='dataframe')
    glm = sm.GLM(response, predictors)
    glm_results = glm.fit()
    glm_sum = glm_results.summary()
    pv = str(glm_sum.tables[1][2][4])
    p_value_g = float(pv)

    return p_value_g


def power(orig_accuracy_list, mutation_accuracy_list):
    eff_size = cohen_d(orig_accuracy_list, mutation_accuracy_list)
    alpha = p_value_glm(orig_accuracy_list, mutation_accuracy_list)
    pow = pw.FTestAnovaPower().solve_power(effect_size=eff_size, nobs=len(orig_accuracy_list) + len(mutation_accuracy_list), alpha=alpha)
    return power

In [14]:
per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/API_Misuse/018")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/API_Misuse/039")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/API_Misuse/057", origin=True)
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/API_Misuse/058", origin=True)
per_qm_daifu = [float(item[2:-2]) for item in per_qm_daifu]
per_qm_origin = [float(item[2:-2]) for item in per_qm_origin]
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/Tensor_Mismatch/007", 'cifar10/')
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/Tensor_Mismatch/081")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-1")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-1")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-1")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-2")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-2")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-2")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/Path_Not_Found/CV-PNF")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/Path_Not_Found/NLP-PNF")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

per_qm_daifu, per_qm_origin = cal_quality_measures("../DLFailureBenchmark/Path_Not_Found/RL-PNF")
if len(set(per_qm_daifu))==len(set(per_qm_origin))==1:
    print(per_qm_daifu[0]==per_qm_origin[0])
else:
    print(p_value_glm(per_qm_origin, per_qm_daifu))
    print(p_value_glm(per_qm_origin, per_qm_daifu)>=0.05)

../DLFailureBenchmark/API_Misuse/018
0.903
True
../DLFailureBenchmark/API_Misuse/039
0.527
True
../DLFailureBenchmark/API_Misuse/057
True
../DLFailureBenchmark/API_Misuse/058
0.074
True
../DLFailureBenchmark/Tensor_Mismatch/007
0.691
True
../DLFailureBenchmark/Tensor_Mismatch/081
0.931
True
../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-1
0.965
True
../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-1
0.489
True
../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-1
0.816
True
../DLFailureBenchmark/GPU_Out_Of_Memory/CV-OOM-2
0.05
True
../DLFailureBenchmark/GPU_Out_Of_Memory/NLP-OOM-2
0.752
True
../DLFailureBenchmark/GPU_Out_Of_Memory/RL-OOM-2
0.177
True
../DLFailureBenchmark/Path_Not_Found/CV-PNF
True
../DLFailureBenchmark/Path_Not_Found/NLP-PNF
0.775
True
../DLFailureBenchmark/Path_Not_Found/RL-PNF
True
